In [ ]:
# -*- coding: utf-8 -*-
#  Copyright 2025 United Kingdom Research and Innovation
#  Copyright 2025 The University of Manchester
#  Copyright 2025 Technical University of Denmark
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Hannah Robarts (UKRI-STFC)
#                   Laura Murgatroyd (UKRI-STFC)
#                   Margaret Duff (UKRI-STFC)

# HDF5 reader

In [ ]:
## Example 1 - 3D parallel beam steel sphere example 

This is taken from the CIL deep dive [05_esrf_pipeline.ipynb](https://github.com/TomographicImaging/CIL-Demos/blob/main/demos/4_Deep_Dives/05_esrf_pipeline.ipynb). All credit should go to those authors. 

It is a work in progress.



## Data format: NXTomo

This example uses dataset tomo_00065 from the TomoBank [[1](https://iopscience.iop.org/article/10.1088/1361-6501/aa9c19)] multidistance dataset. The sample is a steel sphere measured at various propagation distances to demonstrate the effect of propagation based phase contrast imaging.

The tomo_00065.h5 dataset can be retrieved from https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#multi-distance using:

`wget https://g-a0400.fd635.8443.data.globus.org/tomo_00064_to_00067/tomo_00065.h5`

[1] De Carlo, Francesco, et al. “TomoBank: a tomographic data repository for computational x-ray science.” Measurement Science and Technology 29.3 (2018): 034004. http://www.doi.org/10.1088/1361-6501/aa9c19

In [ ]:
# CIL methods
from cil.framework import DataContainer
from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer
from cil.io.utilities import HDF5_utilities
from cil.io import TIFFWriter
from cil.processors import Normaliser, RingRemover, TransmissionAbsorptionConverter, CentreOfRotationCorrector, PaganinProcessor
from cil.recon import FBP
# Additional packages
import numpy as np # conda install numpy
import matplotlib.pyplot as plt # conda install matplotlib
# Custom methods
from readers.hdf5_parallel_reader import HDF5_ParallelDataReader

## CIL Version

This notebook was developed using CIL v25.0.0

Update this filepath to where you have saved the dataset:

In [ ]:
filename = 'tomo_00065.h5'
HDF5_utilities.print_metadata(filename)

We use the generic 'HDF5_ParallelDataReader', set the paths to the required information in the metadata and then read in the data 


In [ ]:
reader = HDF5_ParallelDataReader(filename, 
                                 dataset_path=('exchange/data'),
                                 distance_units='mm', angle_units='degree')
                                

reader.configure_angles(angles_path='exchange/theta', HDF5_units='degree')

reader.configure_pixel_sizes('measurement/instrument/detector/x_actual_pixel_size',
                             'measurement/instrument/detector/y_actual_pixel_size',
                             HDF5_units = 'um')

reader.configure_normalisation_data(flatfield_path='exchange/data_white',
                                    darkfield_path='exchange/data_dark')

# Alternatively, you can set the sample to detector distance directly as a float value rather than passing a path to the value in the HDF5 file
reader.configure_sample_detector_distance(sample_detector_distance=58, HDF5_units='mm') 

data = reader.read()

In [ ]:
islicer(data)


In [ ]:

show_geometry(data.geometry)

Let's normalise the data using the flat and dark fields that were seen in the hdf5 file and read in by the reader. First lets visualise them: 

In [ ]:
show2D([reader.flatfield, reader.darkfield], title=['Flatfield', 'Darkfield'])

We see that show2D has chosen to display slice 1 and slice 2 of the flat and dark field objects. Lets look at the shapes in more detail. 

In [ ]:
print('Flatfield shape, ', reader.flatfield.shape)
print('Darkfield shape, ', reader.darkfield.shape)

There are 4 fla and 4 darks. We take averages of these images and pass the resulting images to the CIL normaliser 

In [ ]:
average_flat = np.mean(reader.flatfield, axis = 0)
average_dark = np.mean(reader.darkfield, axis = 0)
processor = Normaliser(flat_field=average_flat, dark_field=average_dark)
processor.set_input(data)
data_normalised = processor.get_output()

# Use the show2D method to check the effect of the normalisation
show2D([data, data_normalised],
       title=['Before Normalisation', 'After Normalisation'])

In [ ]:
from cil.plugins.tigre import FBP

reconstruction = FBP(acquisition_geometry=data.geometry)(data_normalised)

islicer(reconstruction)

## Example 2 - 2D fan beam multi-material example 

Let's try another dataset! This time a 2D fan beam dataset available on zenodo: 

Khalil, M., Kehres, J., & Mustafa, W. (2023). Hyperspectral 2D fan-beam X-ray CT dataset of 5 materials (1.0.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.8307932. 

Download the data and set the path to the data and, as before, let's view the metadata


In [ ]:
filename = 'sino25_interpol_line.h5'
HDF5_utilities.print_metadata(filename)


There is nothing stored in the hdf5 file :(. However, there is lots of information on the zenodo page that we can use! 

"Hyperspectral X-ray CT dataset acquired at the DTU 3D imaging center. The phantom consists of 5 materials: Aluminium (10 mm) and PVC (7.8 mm) in solid blocks. Sugar, H2O2, and H2O in circular glass containers.

3D array with dimension: 128 x 370 x 258 < channel, angle, horizontal >

 

Detector parameters:

Number of detector pixels: 258 (concatenated from 2 detector modules with 128 pixels each and 2 pixel interpolated across a gap between detectors)

Pixel size: 0.077 cm

Sep=0.153  Pixels' gap length (cm)

det_space=(ndet)*pixel_size+Sep # physical width of detector in cm (pixels*pixel_size), including the gap

 

Acquisition Parameters

360 # Angular span of projections in degrees

370 # Number of projections. note: last projection taken is not a duplicate of the first projection. At angle: 360/370 degrees from first projection.

115.0 # Source-Detector distance in cm

0 # Vertical source shift from perfect placement

0 # Vertical detector shift from perfect placement

57.5 # Source-AxisOfRotation distance in cm

 

rot_axis_x = 0 # x-position offset of AxisOfRotation

rot_axis_y = 0 # y-position offset of AxisOfRotation"

Let's set up the hdf5 reader! This time we use the cone beam version

In [ ]:
from readers.hdf5_cone_reader import HDF5_ConeDataReader

reader = HDF5_ConeDataReader(filename, 
                                 dataset_path=('data'),
                                 distance_units='cm', angle_units='degree', dimension_labels=['channel', 'angle', 'horizontal'])
                                

reader.configure_angles(angles=[ -i*360/370 for i in range(370)], HDF5_units='degree')

reader.configure_pixel_sizes(pixel_size_x=0.077,
                             HDF5_units = 'cm')

reader.configure_source_detector_distance(source_detector_distance=115, HDF5_units='cm')
reader.configure_sample_detector_distance(sample_detector_distance=115-57, HDF5_units='cm')

reader.configure_channels(num_channels=128)
data = reader.read()

In [ ]:
islicer(data)

In [ ]:
show_geometry(data.geometry)

In [ ]:
reconstruction = data.geometry.get_ImageGeometry().allocate(0)

for i in range(0,128,10):
    single_data = data.get_slice(channel=i)
    print(single_data)
    if i==0:
        recon = FBP(acquisition_geometry=single_data.geometry)
    show2D(recon(single_data), title=f'Channel {i}', fix_range=(0,4))